# Exploring Table Data: 2014-15 NBA Shot Log

Erik Gabrielsen, Danh Nguyen

## Business Understanding



## Data Understanding

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('~/Downloads/shot_logs.csv') # read in the csv file

df.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148


In [4]:
df.describe()

/Users/danh/anaconda/envs/machinelearning/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,GAME_ID,FINAL_MARGIN,SHOT_NUMBER,PERIOD,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_id
count,1.280690e+05,128069.000000,128069.000000,128069.000000,122502.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000
mean,2.140045e+07,0.208723,6.506899,2.469427,12.453344,2.023355,2.765901,13.571504,2.264670,159038.487284,4.123015,0.452139,0.997314,157238.251247
std,2.578773e+02,13.233267,4.713260,1.139919,5.763265,3.477760,3.043682,8.888964,0.441159,78791.172947,2.756446,0.497706,1.130978,79362.389336
min,2.140000e+07,-53.000000,1.000000,1.000000,0.000000,0.000000,-163.600000,0.000000,2.000000,708.000000,0.000000,0.000000,0.000000,708.000000
25%,2.140023e+07,-8.000000,3.000000,1.000000,NaN,0.000000,0.900000,4.700000,2.000000,101249.000000,2.300000,0.000000,0.000000,101162.000000
50%,2.140045e+07,1.000000,5.000000,2.000000,NaN,1.000000,1.600000,13.700000,2.000000,201949.000000,3.700000,0.000000,0.000000,201939.000000
75%,2.140067e+07,9.000000,9.000000,3.000000,NaN,2.000000,3.700000,22.500000,3.000000,203079.000000,5.300000,1.000000,2.000000,202704.000000
max,2.140091e+07,53.000000,38.000000,7.000000,24.000000,32.000000,24.900000,47.200000,3.000000,530027.000000,53.200000,1.000000,3.000000,204060.000000


In [4]:
print(df.dtypes)
print(df.info())


GAME_ID                         int64
MATCHUP                        object
LOCATION                       object
W                              object
FINAL_MARGIN                    int64
SHOT_NUMBER                     int64
PERIOD                          int64
GAME_CLOCK                     object
SHOT_CLOCK                    float64
DRIBBLES                        int64
TOUCH_TIME                    float64
SHOT_DIST                     float64
PTS_TYPE                        int64
SHOT_RESULT                    object
CLOSEST_DEFENDER               object
CLOSEST_DEFENDER_PLAYER_ID      int64
CLOSE_DEF_DIST                float64
FGM                             int64
PTS                             int64
player_name                    object
player_id                       int64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128069 entries, 0 to 128068
Data columns (total 21 columns):
GAME_ID                       128069 non-null int64
MATCHUP                  

In [5]:
pd.isnull(df['SHOT_CLOCK']).sum()

5567

In [6]:
df.count()

GAME_ID                       128069
MATCHUP                       128069
LOCATION                      128069
W                             128069
FINAL_MARGIN                  128069
SHOT_NUMBER                   128069
PERIOD                        128069
GAME_CLOCK                    128069
SHOT_CLOCK                    122502
DRIBBLES                      128069
TOUCH_TIME                    128069
SHOT_DIST                     128069
PTS_TYPE                      128069
SHOT_RESULT                   128069
CLOSEST_DEFENDER              128069
CLOSEST_DEFENDER_PLAYER_ID    128069
CLOSE_DEF_DIST                128069
FGM                           128069
PTS                           128069
player_name                   128069
player_id                     128069
dtype: int64

In [7]:
df.CLOSEST_DEFENDER.unique()

array(['Anderson, Alan', 'Bogdanovic, Bojan', 'Brown, Markel',
       'Young, Thaddeus', 'Williams, Deron', 'Jack, Jarrett',
       'Plumlee, Mason', 'Morris, Darius', 'Ellington, Wayne',
       'Lin, Jeremy', 'Hill, Jordan', 'Green, Willie', 'Smart, Marcus',
       'Young, James', 'Jerebko, Jonas', 'Crowder, Jae', 'Thomas, Isaiah',
       'Brooks, Aaron', 'Hinrich, Kirk', 'Gibson, Taj', 'Butler, Jimmy',
       'Noah, Joakim', 'Harris, Devin', 'Barea, Jose Juan',
       'Augustin, D.J.', 'Drummond, Andre', 'Lucas III, John',
       'Dinwiddie, Spencer', 'Miles, CJ', 'Hill, George', 'Watson, CJ',
       'Whittington, Shayne', 'West, David', 'Hill, Solomon',
       'Frazier, Tim', 'Noel, Nerlens', 'Grant, Jerami', 'Sampson, JaKarr',
       'Mbah a Moute, Luc', 'Wall, John', 'Gortat, Marcin',
       'Temple, Garrett', 'Pierce, Paul', 'Humphries, Kris',
       'Butler, Rasual', 'Porter, Otto', 'Nene', 'Gallinari, Danilo',
       'Nurkic, Jusuf', 'Lawson, Ty', 'Hickson, JJ', 'Parker, Tony',